In [11]:
from IPython.display import display

In [2]:
import numpy as np
import random

NUM_TRAIN = 10000

In [3]:
tfidf = np.load('doc_tfidf1.npy')

In [32]:
program = tfidf[:227075]
questions = tfidf[227075:]

In [5]:
def generate_training_data():
    Xs, Ys = [], []
    
    for i in range(NUM_TRAIN):
        pos_or_neg = random.randint(0,1)
        
        if pos_or_neg is 1:
            line_id = random.randint(0, program.shape[0]-1)
            
            Xs.append([program[line_id], program[line_id+1]])
            Ys.append(1)
            
        else:
            line1_id = random.randint(0, program.shape[0]-1)
            line2_id = random.randint(0, program.shape[0]-1)
            
            Xs.append([program[line1_id], program[line2_id]])
            Ys.append(0)
            
    return Xs, Ys

In [6]:
from sklearn.model_selection import train_test_split

X, y = generate_training_data()

X_ = []

for t in X:
    X_.append([x+y for x, y in zip(t[0], t[1])])


X_ = np.stack(X_, axis=0)
print (X_)

X_train, X_valid, y_train, y_valid = train_test_split(X_, y, test_size=0.3, random_state=1)

[[ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 ..., 
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]]


In [7]:
print (X_train.shape)

(7000, 10171)


In [8]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
sc.fit(X_train)
X_train_std = sc.transform(X_train)
X_valid_std = sc.transform(X_valid)

In [9]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

svm_rbf = SVC(kernel='rbf', random_state=0, C=10.0, gamma=0.2)
svm_rbf.fit(X_train_std, y_train)

y_pred = svm_rbf.predict(X_valid_std)

print ("Accuracy: %.3f" % (accuracy_score(y_valid, y_pred)))


Accuracy: 0.509


In [39]:
print (questions.shape)
print (X_valid_std.shape)

(3500, 10171)
(3000, 10171)


In [47]:
q_test = []
for q_id in range(500):
    question = questions[q_id*7]
    for i in range(1,7):
        ans = questions[q_id*7 + i]
        set_ = [x+y for x, y in zip(question, ans)]
        set_ = np.array(set_)
        result = svm_rbf.predict(set_.reshape(1,-1))
        if result == 1:
            q_test.append(i)
            break
        elif (i == 6 and result == 0):
            q_test.append(random.randint(1,6))

In [50]:
for i in q_test:
    print (i, end=" ")

1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 

In [ ]:
from sklearn.linear_model import LogisticRegression

log_r = LogisticRegression(C=1000.0, random_state=0)
log_r.fit(X_train_std, y_train)
y_pred_lr = log_r.predict(X_valid_std)

print ("Accuracy: %.3f" % (accuracy_score(y_valid, y_pred_lr)))

In [ ]:
from sklearn.ensemble import RandomForestClassifier
forest = RandomForestClassifier(criterion='entropy',
                                n_estimators=50, 
                                random_state=1,
                                n_jobs=2)

forest.fit(X_train_std, y_train)
y_pred_f = forest.predict(X_valid_std)
print('Accuracy: %.3f' % accuracy_score(y_valid, y_pred_f))

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier

tree = DecisionTreeClassifier(criterion='entropy', max_depth=1)
ada = AdaBoostClassifier(base_estimator=tree, n_estimators=500)
ada = ada.fit(X_train, y_train)
y_pred_ada = ada.predict(X_valid_std)
print('Accuracy: %.3f' % accuracy_score(y_valid, y_pred_ada))